In [81]:
import requests
from bs4 import BeautifulSoup
import re
from pymongo import MongoClient
from datetime import datetime, date
import os
from dotenv import load_dotenv
load_dotenv()

True

In [82]:
url = os.getenv("MONGO_URL")
client = MongoClient(url)
db = client.get_database("kuwtk")

In [83]:
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.text)
    raise Exception(response)

# Kardashian API content

## IMBD Episodes
For each episode: title, imbd_rate, description, season, watch_platforms, air_date, imbd_id

In [84]:
imbd_episodes = db.imbd_episodes

In [113]:
def season_url(
    season): return f"https://www.imdb.com/title/tt1086761/episodes?season={season}"

In [114]:
def parseDate(date):
    if date != "":
        try:
            return datetime.strptime(date, '%d %b. %Y')
        except Exception as error:
            return datetime.strptime(date, '%d %b %Y')
    return None

In [115]:
def getSeasonEpisodeData(html):
    data = []
    list_episodes = html.select("div.list.detail.eplist > div")
    for episode in list_episodes:
        imbd_id = episode.select_one("div.image > a > div").get("data-const")
        image_url_raw = episode.select_one("div.image > a > div > img")
        image_url = None
        if image_url_raw:
            image_url = image_url_raw.get("src")
        s, e = episode.select_one(
            "div.image > a > div > div").get_text().split(",")
        season_num = int(s.replace("S", ""))
        season_episode_num = int(e.replace("Ep", ""))

        air_date_raw = episode.select_one(
            "div.info > div.airdate")
        air_date = None
        raw_air_date = None
        if air_date_raw:
            raw_air_date = air_date_raw.get_text().replace("\n", "").strip()
            air_date = parseDate(raw_air_date)

        title = episode.select_one("div.info > strong > a").get("title")

        imbd_rate_raw = episode.select_one(
            "div.info span.ipl-rating-star__rating")
        imbd_rate = None
        if imbd_rate_raw:
            imbd_rate = float(imbd_rate_raw.get_text())

        imbd_rate_votes_raw = episode.select_one(
            "div.info span.ipl-rating-star__total-votes")
        imbd_rate_votes = None
        if imbd_rate_votes_raw:
            imbd_rate_votes = int(re.sub(
                r"([()])", "", imbd_rate_votes_raw.get_text()))

        description_raw = episode.select_one(
            "div.info > div[itemprop='description']")
        description = None
        if description_raw:
            description = description_raw.get_text().replace("\n", "")

        watch_links = []
        data.append(
            {
                "imbd_id": imbd_id,
                "image_url": image_url,
                "season": season_num,
                "episode": season_episode_num,
                "raw_air_date": raw_air_date,
                "air_date": air_date,
                "title": title,
                "imbd_rate": imbd_rate,
                "imbd_rate_votes": imbd_rate_votes,
                "description": description,
                "watch_links": watch_links
            }
        )
    return data

In [124]:
errors = []
for s in range(1, 21):
    print(s)
    try:
        html = get_html(season_url(s))
        docs = getSeasonEpisodeData(html)
        [imbd_episodes.update_one({"imbd_id": doc["imbd_id"]}, {"$set": doc}, upsert=True)
         for doc in docs]
    except Exception as error:
        errors.append(error)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [125]:
errors

[]

In [126]:
h = imbd_episodes.aggregate([
    {
        "$group": {
            "_id": "$season",
            "totalEpisodes": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
])

In [127]:
[e for e in h]

[{'_id': 1, 'totalEpisodes': 8},
 {'_id': 2, 'totalEpisodes': 11},
 {'_id': 3, 'totalEpisodes': 12},
 {'_id': 4, 'totalEpisodes': 11},
 {'_id': 5, 'totalEpisodes': 12},
 {'_id': 6, 'totalEpisodes': 16},
 {'_id': 7, 'totalEpisodes': 19},
 {'_id': 8, 'totalEpisodes': 21},
 {'_id': 9, 'totalEpisodes': 20},
 {'_id': 10, 'totalEpisodes': 20},
 {'_id': 11, 'totalEpisodes': 13},
 {'_id': 12, 'totalEpisodes': 26},
 {'_id': 13, 'totalEpisodes': 14},
 {'_id': 14, 'totalEpisodes': 20},
 {'_id': 15, 'totalEpisodes': 16},
 {'_id': 16, 'totalEpisodes': 12},
 {'_id': 17, 'totalEpisodes': 12},
 {'_id': 18, 'totalEpisodes': 6},
 {'_id': 19, 'totalEpisodes': 9},
 {'_id': 20, 'totalEpisodes': 14}]

## Wikipedia Episodes
For each episode: title, imbd_rate, description, season, watch_platforms, air_date, imbd_id

In [5]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_Keeping_Up_with_the_Kardashians_episodes"

In [8]:
html = get_html(wiki_url)
wiki_ep = html.select("tr.vevent")

In [97]:
episodes_wikipedia = []
for e in wiki_ep:
    episode_overall = int(e.select_one("tr > th:first-child").get_text())
    episode_data = [i.get_text() for i in e.select("tr > td")]
    episode = episode_data[0]
    title = episode_data[1]
    special_episode = False
    air_date = None
    us_viewers = None
    season = None
    if len(episode_data) == 5:
        episode = episode_overall
        episode_overall = None
        season = episode_data[0]
        special_episode = True
        air_date = datetime.fromisoformat(
            re.sub(r"([()])", "", re.search(r"\((.*?)\)", episode_data[3]).group()))
        if episode_data[4] != "N/A":
            us_viewers = float(re.sub(
                r"\[[^)]*\]", "", episode_data[4]).replace('″', ""))
    if len(episode_data) <= 4:
        air_date = datetime.fromisoformat(
            re.sub(r"([()])", "", re.search(r"\((.*?)\)", episode_data[2]).group()))
        if len(episode_data) == 4 and episode_data[3] != "N/A":
            us_viewers = float(re.sub(
                r"\[[^)]*\]", "", episode_data[3]).replace('″', ""))
    episodes_wikipedia.append({
        "title": title.strip('"'),
        "episode": int(episode),
        "season": season,
        "episode_overall": episode_overall,
        "air_date": air_date,
        "us_viewers": us_viewers,
        "special_episode": special_episode
    })

In [98]:
wiki_episodes = db.wiki_episodes

In [101]:
errors = []
for doc in episodes_wikipedia:
    print(doc["episode_overall"])
    try:
        wiki_episodes.update_one({"title": doc["title"]}, {
                                 "$set": doc}, upsert=True)
    except Exception as error:
        errors.append(error)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
132
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
231
232
233
234
235
236
237
238
239
240
241
242
240
241
242
243
244
245
246
247
248
249
250
251
255
256
257
258
259
260
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [100]:
errors

[]